##### 问题:
我们使用访问者模式来遍历一个深度嵌套的树结构，但由于超出了 Python 的递归限制
而崩溃。我们想要去掉递归，但依旧保持访问者模式的编程风格。

##### 解决方案:
巧妙利用生成器有时候可用来消除树的遍历或查找算法中的递归。在 8.21 节中，我们
已经给出了一个访问者类。下面是这个类的另一种实现方式，通过堆栈和生成器来驱
动计算，完全不使用递归。

In [28]:
import types


class Node:
    pass


class NodeVisitor:
    def visit(self, node):
        stack = [node]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result

    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)

    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format(
            'visit_' + type(node).__name__))


如果使用这个类，就会发现配合之前已有的代码（可能使用了递归），程序仍然可以正
常工作。实际上，我们可以用其替换上一节中的访问者类实现。例如，考虑下面的代
码，其中涉及表达式树：

In [29]:
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand


class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Add(BinaryOperator):
    pass


class Sub(BinaryOperator):
    pass


class Mul(BinaryOperator):
    pass


class Div(BinaryOperator):
    pass


class Negate(UnaryOperator):
    pass


class Number(Node):
    def __init__(self, value):
        self.value = value

# A sample visitor class that evaluates expressions


class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value

    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)

    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)

    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)

    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)

    def visit_Negate(self, node):
        return -self.visit(node.operand)


if __name__ == '__main__':
    # 1 + 2*(3-4) / 5
    t1 = Sub(Number(3), Number(4))
    t2 = Mul(Number(2), t1)
    t3 = Div(t2, Number(5))
    t4 = Add(Number(1), t3)
    # Evaluate it
    e = Evaluator()
    print(e.visit(t4))  # Outputs 0.6

0.6


上述代码在处理简单的表达式时是没有问题的。但是，Evaluator 的实现中使用了递归，
如果嵌套层次太深的话程序就会崩溃。示例如下：

In [30]:
a = Number(0)
for n in range(1, 100000):
    a = Add(a, Number(n))
e = Evaluator() 
# e.visit(a) 



<img src="3.png" width="140% ">

现在，我们把 Evaluator 类稍微修改一下：

In [31]:
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    def visit_Add(self, node):
        yield (yield node.left) + (yield node.right)
    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)
    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)
    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)
    def visit_Negate(self, node):
        yield -(yield node.operand)



如果再次尝试同样的试验，会发现程序突然就可以正常工作了，真是神奇！

In [32]:
a = Number(0)
for n in range(1,100000):
    a = Add(a, Number(n))
e = Evaluator() 
e.visit(a)

4999950000

如果想在任意一个方法中添加自定义的处理，程序依然可以正常工作。示例如下：

In [33]:

class Evaluator(NodeVisitor):
    def visit_Add(self, node):
        print('Add:', node)
        lhs = yield node.left   
        print('left=', lhs)
        rhs = yield node.right
        print('right=', rhs)
        yield lhs + rhs 
    def visit_Number(self, node):
        return node.value
    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)
    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)
    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)
    def visit_Negate(self, node):
        yield -(yield node.operand)



下面是示例输出：

In [34]:
e = Evaluator()
e.visit(t4) 

Add: <__main__.Add object at 0x0000015C5E2E8940>
left= 1
right= -0.4


0.6

本节很好地展示了如何利用生成器和协程来控制程序的执行流。这种令人费解的技巧
常常能带来很大的优势。要理解本节的内容，需要深入了解几个要点。

首先，在有关遍历树结构的问题中，为了避免使用递归，常见的策略就是利用栈或者
队列来实现算法。例如，深度优先遍历完全可以实现为将第一个遇到的节点压入栈中，
一旦处理结束再将其弹出。解决方案中给出的 visit()方法的核心就是按照这个思路实现
的。算法一开始会将初始节点压入 stack 列表中（这里的栈以 Python 列表的形式来
实现），然后继续运行直到栈为空为止。在执行算法的时候，栈会根据树结构的深度进
行增长。


第二个要点在于生成器中 yield 语句的行为。当遇到 yield 语句时，生成器会产生出一
个值然后暂停执行。本节正是利用这个特性来取代递归。例如，现在我们不用像这样
编写递归式的表达式了：


value = self.visit(node.left)


我们用下面这条语句来替代：


value = yield node.left


在幕后，这条语句会将 node.left 节点发送回给 visit()方法。之后，visit()就可以为该节
点调用合适的 visit_Name()方法了。从某种意义上说，这几乎和递归算法恰好相反。也
就是说，现在不是通过递归调用 visit()来遍历树节点了，而是在处理的过程中用 yield
语句来暂停计算。因此，yield 本质上可当做一种信号来告诉算法当前处在 yield 状态的
节点需要先被处理，之后剩下的处理才可以继续进行。


本节中最后一个需要考虑的问题是如何传递结果。当我们使用生成器函数时，我们不能
再使用 return 语句来发送结果了（这么做会产生 SyntaxError 异常）。因此，yield 语句
必须来承担这个责任。在本节中，如果由 yield 语句产生出的值是非节点类型（non-Node
type）的，则认为该值是要发送给计算过程中的下一个步骤的。这正是在代码中使用变
量 last_return 的目的所在。一般来说，last_return 将保存某个访问方法上一次产生出的
值。这个值会作为 yield 语句的返回值发送到上一个执行的方法中。例如，在下面的代
码中：


value = yield node.left


变量 value 将获得 last_return 的值，而这个值正是在为节点 node.left 调用访问方法时返
回的结果。

以上所有要点都可以在下面的代码片段中找到：

In [ ]:
'''
try:
    last = stack[-1]
    if isinstance(last, types.GeneratorType):
        stack.append(last.send(last_result))
        last_result = None
    elif isinstance(last, Node):
        stack.append(self._visit(stack.pop()))
    else:
        last_result = stack.pop()
except StopIteration:
    stack.pop()
'''

这段代码简单地查看栈顶并决定下一步该做什么。如果是生成器，那么就调用它的 send()
方法将上次得到的结果（如果有结果的话）添加到栈中以待后续处理。send()返回的值
和传给 yield 语句的值是相同的。因此，在 yield node.left 这样的语句中，send()返回的
就是 Node 的实例 node.left，并会将其放置在栈的顶部。


如果栈顶是一个 Node 实例，那么该实例会被替换为在该节点上调用合适的访问方法所
得到的结果。正是因为这样，我们完全避免了对递归的使用。之前我们是在各个访问
方法中以递归的方式直接调用 visit()（参见上一节解决方案中的实现），现在不必这么
做了。只要在各个访问方法中使用 yield，那么程序就能正常工作。


最后，如果栈顶元素为其他值，则可认为这是某种类型的返回值。我们将其从栈中弹
出然后保存到 last_result 中。如果栈中的下一个元素是生成器，那么就将它作为 yield
语句的返回值发送出去。应该提到的是，visit()的最后一个返回值也会赋给 last_result。
这样就使得本节中的代码也能适用于传统的递归实现。如果没有用到生成器，last_result
就保存着代码中 return 语句的返回值。


本节中一个潜在的危险在于产生 Node 和非 Node 值之间的区别。在我们的实现中会自
动遍历所有的 Node 实例。这意味着我们不能把 Node 当做返回值来进行传递。在实践
中，这也许无关紧要。但是如果确实有这个需求，就需要对算法做轻微的调整。例如，
可以通过引入另一个类来解决：

In [ ]:
class Visit:
    def __init__(self, node):
        self.node = node
class NodeVisitor:
    def visit(self, node):
        stack = [ Visit(node) ]
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Visit):
                    stack.append(self._visit(stack.pop().node))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result
    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format('visit_' + type(node).__name__)) 

根据上面的实现，现在访问方法看起来就是这样的了：

In [ ]:
class Evaluator(NodeVisitor):

    def visit_Add(self, node):
        yield (yield Visit(node.left)) + (yield Visit(node.right))
    def visit_Sub(self, node):
        yield (yield Visit(node.left)) - (yield Visit(node.right))

看过本节之后，你可能会倾向于去实现一种不涉及 yield 的解决方案。但是，这么做会
使得我们必须在代码中处理本节中已经提到过的诸多问题。例如，要消除对递归的使
用，需要维护一个栈。也需要有某种方法来管理对树结构的遍历以及调用各种访问者
方法的逻辑。没有生成器的帮助，这种代码将演变成一锅大杂烩，其中混杂着对栈的
操作、回调函数以及其他的组件。坦白说，使用 yield 的主要优势在于我们能够以优雅
的风格编写出非递归式的代码，而且看起来和递归式的实现几乎一样。

In [ ]:
#最终代码：
import types
class Node:
    pass
class NodeVisitor:
    def visit(self, node):
        stack = [node]
        print(type(stack))
        last_result = None
        while stack:
            try:
                last = stack[-1]
                if isinstance(last, types.GeneratorType):
                    stack.append(last.send(last_result))
                    last_result = None
                elif isinstance(last, Node):
                    stack.append(self._visit(stack.pop()))
                else:
                    last_result = stack.pop()
            except StopIteration:
                stack.pop()
        return last_result

    def _visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)

    def generic_visit(self, node):
        raise RuntimeError('No {} method'.format(
            'visit_' + type(node).__name__))
class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand


class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right


class Add(BinaryOperator):
    pass


class Sub(BinaryOperator):
    pass


class Mul(BinaryOperator):
    pass


class Div(BinaryOperator):
    pass


class Negate(UnaryOperator):
    pass


class Number(Node):
    def __init__(self, value):
        self.value = value

class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    def visit_Add(self, node):
        yield (yield node.left) + (yield node.right)
    def visit_Sub(self, node):
        yield (yield node.left) - (yield node.right)
    def visit_Mul(self, node):
        yield (yield node.left) * (yield node.right)
    def visit_Div(self, node):
        yield (yield node.left) / (yield node.right)
    def visit_Negate(self, node):
        yield -(yield node.operand)

a = Number(0)
for n in range(1,6):
    a = Add(a, Number(n))
e = Evaluator() 
print(e.visit(a))

